In [1]:
import pathlib
import random
import string
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Bidirectional,GRU,LSTM,Embedding
from tensorflow.keras.layers import Dense,MultiHeadAttention,LayerNormalization,Embedding,Dropout,Layer
from tensorflow.keras import Sequential,Input
from tensorflow.keras.callbacks import ModelCheckpoint
import os 
from nltk.translate.bleu_score import sentence_bleu

In [2]:
    # %cd C:/Users/Lenovo/Desktop/CODING/major_project/parallel-corpus/sanskrit-english
    # %ls

In [3]:
file_directory = r'C://Users//sampa//Downloads/sanskrit-english-20240406T035746Z-001/sanskrit-english'

import os

# Function to read lines from a file and remove newline characters
def read_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

# Paths to English and Sanskrit files
english_files = [
    file_directory  +'//bhagvadgita_english.txt',
    file_directory  +'//bible_english.txt',
    file_directory  +'//manu_english.txt',
    file_directory  +'//ramayan_english.txt',
    file_directory  +'//rigveda_english.txt'
]

sanskrit_files = [
    file_directory  +'//bhagvadgita_sanskrit.txt',
    file_directory  +'//bible_sanskrit.txt',
    file_directory  +'//manu_sanskrit.txt',
    file_directory  + '//ramayan_sanskrit.txt',
    file_directory  +'//rigveda_sanskrit.txt'
]

# Read English and Sanskrit lines from respective files
eng_lines = []
sanskrit_lines = []

for eng_file, sanskrit_file in zip(english_files, sanskrit_files):
    eng_lines.extend(read_lines(os.path.join(file_directory, eng_file)))
    sanskrit_lines.extend(read_lines(os.path.join(file_directory, sanskrit_file)))

# Create pairs of English and Sanskrit sentences
sentence_pairs = []

for eng_sentence, sanskrit_sentence in zip(eng_lines, sanskrit_lines):
    sentence_pairs.append((eng_sentence, '[start] ' + sanskrit_sentence + ' [end]'))

print("Number of sentence pairs created:", len(sentence_pairs))


Number of sentence pairs created: 34374


In [4]:
import random
print(random.choice(sentence_pairs))

('lightly with piercing ends as \x92twere two ranks of heroes ranged for fight.', '[start] नेमधितान पौंस्या वर्थेव विष्टान्ता| [end]')


In [5]:
import pandas as pd

# Assuming 'sentence_pairs' contains pairs of English and Sanskrit sentences
# Split the pairs into two lists: one for English and one for Sanskrit
english_sentences = [pair[0] for pair in sentence_pairs]
sanskrit_sentences = [pair[1] for pair in sentence_pairs]

# Create a DataFrame with these lists
df = pd.DataFrame({
    'English': english_sentences,
    'Sanskrit': sanskrit_sentences
})
# Remove '[start]' and '[end]' tokens from the 'Sanskrit' column
df['Sanskrit'] = df['Sanskrit'].str.replace('\[start\] ', '')  # Remove '[start]' and the following space
df['Sanskrit'] = df['Sanskrit'].str.replace(' \[end\]', '')  # Remove '[end]' and the preceding space

# Display the first few rows of the DataFrame to verify the changes
print(df.head())



                                             English  \
0  Dhrtarastra said O Sanjaya ! What did my men a...   
1  Sanjaya said Seeing the army of the sons of Pa...   
2  O teacher ! Behold this mighty army of the son...   
3  The heroes and mighty archers, comparable in w...   
4  Dhrstaketu, Cekitana and the valourous king of...   

                                            Sanskrit  
0  [start] धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत...  
1  [start] सञ्जय उवाच दृष्ट्वा तु पाण्डवानीकं व्य...  
2  [start] पश्यैतां पाण्डुपुत्राणामाचार्य महतीं च...  
3  [start] अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।...  
4  [start] धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवा...  


In [6]:
df

,English,Sanskrit
0,Dhrtarastra said O Sanjaya ! What did my men a...,[start] धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत...
1,Sanjaya said Seeing the army of the sons of Pa...,[start] सञ्जय उवाच दृष्ट्वा तु पाण्डवानीकं व्य...
2,O teacher ! Behold this mighty army of the son...,[start] पश्यैतां पाण्डुपुत्राणामाचार्य महतीं च...
3,"The heroes and mighty archers, comparable in w...",[start] अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।...
4,"Dhrstaketu, Cekitana and the valourous king of...",[start] धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवा...
...,...,...
34369,and sought the somapourers home.,[start] अगछःसोमिनो गर्हम| [end]
34370,venya that mortal man hast thou for Āstrabudhn...,[start] तवं तयमिन्द्र मर्त्यमास्त्रबुध्नाय वेन...
34371,o indra many a time set free.,[start] मुहुःश्रथ्ना मनस्यवे| [end]
34372,bring indra to the east again that sun who now...,[start] तवं तयमिन्द्र सूर्यं पश्चा सन्तं पुरस ...


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize Sanskrit sentences
sanskrit_tokenizer = Tokenizer(char_level=False)
sanskrit_tokenizer.fit_on_texts(df['Sanskrit'])
sanskrit_sequences = sanskrit_tokenizer.texts_to_sequences(df['Sanskrit'])

# Tokenize English sentences
english_tokenizer = Tokenizer(char_level=False, filters='')
english_tokenizer.fit_on_texts(df['English'])
english_sequences = english_tokenizer.texts_to_sequences(df['English'])

# Vocabulary sizes
sanskrit_vocab_size = len(sanskrit_tokenizer.word_index) + 1
english_vocab_size = len(english_tokenizer.word_index) + 1

# Padding sequences
sanskrit_padded = pad_sequences(sanskrit_sequences, padding='post')
english_padded = pad_sequences(english_sequences, padding='post')

# Decoder Input Data
decoder_input_data = english_padded[:, :-1]

# Decoder Target Data
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]


In [8]:
import numpy as np

def one_hot_encode(sequences, num_classes, dtype=np.float32):
    # ... (rest of the function)
    one_hot_output = np.zeros((len(sequences), max(len(sequence) for sequence in sequences), num_classes), dtype=dtype)
    # ... (rest of the function)


In [9]:
def one_hot_encode_sparse(sequences, num_classes, dtype=np.float32):
    rows = []
    cols = []
    data = []
    max_sequence_length = max(len(sequence) for sequence in sequences)
    
    for i, sequence in enumerate(sequences):
        for t, word_index in enumerate(sequence):
            if word_index > 0:  # Skip 0 padding
                rows.append(i)
                cols.append(t)
                data.append(1.0)
    
    one_hot_output = csr_matrix((data, (rows, cols)), shape=(len(sequences), max_sequence_length, num_classes), dtype=dtype)
    return one_hot_output
decoder_target_one_hot = one_hot_encode(decoder_target_data, english_vocab_size)


MemoryError: Unable to allocate 1.24 TiB for an array with shape (34374, 226, 43947) and data type float32

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

latent_dim = 256  # Dimensionality of the encoding space

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=sanskrit_vocab_size, output_dim=latent_dim)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=latent_dim)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_inputs), initial_state=encoder_states)
decoder_dense = Dense(english_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    32082432    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    11250432    ['input_2[0][0]']                
                                                                                              

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Switch to sparse_categorical_crossentropy and provide integer targets
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Make sure your target data is integer-coded, not one-hot encoded
decoder_target_data = np.argmax(decoder_target_one_hot, axis=-1)

model.fit([sanskrit_padded, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=100,
          validation_split=0.2)


Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'model/dense/Tensordot/MatMul' defined at (most recent call last):
    File "c:\Users\sampa\anaconda3\envs\ml\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\sampa\anaconda3\envs\ml\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\sampa\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\sampa\anaconda3\envs\ml\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\sampa\anaconda3\envs\ml\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\sampa\anaconda3\envs\ml\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\sampa\anaconda3\envs\ml\lib\asyncio\ba

In [ ]:
# model.fit([sanskrit_padded, decoder_input_data], decoder_target_one_hot,
#           batch_size=32,
#           epochs=100,  # Use an appropriate number of epochs
#           validation_split=0.2)


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)


In [ ]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)
